In [1]:
import numpy as np
from multiprocessing import Pool

In [2]:
#!mkdir clean_files
#!mkdir html_files

In [3]:
import pickle
import requests
from bs4 import BeautifulSoup

def download_document(pid):
    """ Download and process a Habr document and its comments """
    # выгрузка документа
    r = requests.get('https://habrahabr.ru/post/' +str(pid) + '/')
    # парсинг документа
    soup = BeautifulSoup(r.text, 'html5lib') # instead of html.parser
    doc = {}
    doc['id'] = pid
    if not soup.find("span", {"class": "post__title-text"}):
        # такое бывает, если статья не существовала или удалена
        doc['status'] = 'title_not_found'
    else:
        doc['status'] = 'ok'
        doc['title'] = soup.find("span", {"class": "post__title-text"}).text
        doc['text'] = soup.find("div", {"class": "post__text"}).text
        doc['time'] = soup.find("span", {"class": "post__time"}).text
        doc['hubs'] = soup.find("ul", {"class": "post__hubs"}).text
        doc['likes'] = soup.find("span", {"class": "voting-wjt__counter"}).text
    
    if 'hubs' in doc:    
        # сохранение результата в отдельный файл
        clean_filename = r'clean_files/' + str(pid) + '.pkl'
        with open(clean_filename, 'wb') as f:
            pickle.dump(doc, f)
        
        html_filename = r'html_files/' + str(pid) + '.pkl'
        with open(html_filename, 'w') as html_out:
            html_out.write(r.text)

In [ ]:
with Pool(10) as p:
    docs = p.map(download_document, np.arange(313000, 500000))